In [1]:
import pickle
import turicreate as tc
import pandas as pd
import numpy as np

In [2]:
import os
os.chdir('../data-import/')
if 0:
    %run main.py 
else:
    data = pickle.load( open( "london_poi.pickle", "rb" ) )
os.chdir('../recommender/')

In [3]:
df = pd.DataFrame.from_dict(data)

In [4]:
df.head()

,category,geoCode,name,subType,tags,type
0,SIGHTS,"{'latitude': 51.496395, 'longitude': -0.171935}",Victoria and Albert Museum,POINT_OF_INTEREST,"[sightseeing, artgallerie, museum, sights, lan...",location
1,SIGHTS,"{'latitude': 51.500996, 'longitude': -0.177414}",Royal Albert Hall,POINT_OF_INTEREST,"[sightseeing, sights, landmark, historicplace]",location
2,SIGHTS,"{'latitude': 51.496143, 'longitude': -0.174409}",Natural History Museum,POINT_OF_INTEREST,"[sightseeing, restaurant, museum, cheap]",location
3,SIGHTS,"{'latitude': 51.499092, 'longitude': -0.162348}",Harrods,POINT_OF_INTEREST,"[sightseeing, shopping, restaurant]",location
4,SIGHTS,"{'latitude': 51.506935, 'longitude': -0.179232}",Kensington Gardens,POINT_OF_INTEREST,"[sightseeing, sights, landmark, historicplace,...",location


In [5]:
df2 = df[ ['category', 'tags']]
df2['index'] = df2.index

/Users/Devansh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
df2.head()

,category,tags,index
0,SIGHTS,"[sightseeing, artgallerie, museum, sights, lan...",0
1,SIGHTS,"[sightseeing, sights, landmark, historicplace]",1
2,SIGHTS,"[sightseeing, restaurant, museum, cheap]",2
3,SIGHTS,"[sightseeing, shopping, restaurant]",3
4,SIGHTS,"[sightseeing, sights, landmark, historicplace,...",4


In [7]:
item_data = tc.SFrame(df2)

In [8]:
item_data

category,tags,index
SIGHTS,"[sightseeing,artgallerie, museum, ...",0
SIGHTS,"[sightseeing, sights,landmark, historicplace] ...",1
SIGHTS,"[sightseeing, restaurant,museum, cheap] ...",2
SIGHTS,"[sightseeing, shopping,restaurant] ...",3
SIGHTS,"[sightseeing, sights,landmark, historicplace, ...",4
SIGHTS,"[sightseeing, museum]",5
RESTAURANT,"[restaurant, nightlife]",6
SIGHTS,"[sightseeing, sights,park, landmark] ...",7
RESTAURANT,[restaurant],8
RESTAURANT,"[shopping, restaurant,vegetarian] ...",9


In [9]:
m = tc.recommender.item_content_recommender.create(item_data, item_id = 'index', verbose=True)

Applying transform:
Class             : AutoVectorizer

Model Fields
------------
Features          : ['category', 'tags']
Excluded Features : ['index']

Column    Type  Interpretation  Transforms  Output Type
--------  ----  --------------  ----------  -----------
category  str   categorical     None        str        
tags      list  categorical     Flatten     dict       


Defaulting to brute force instead of ball tree because there are multiple distance components.


Starting brute force nearest neighbors model training.

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

...

+--------------+---------+-------------+--------------+

Preparing data set.

Data has 0 observations with 0 users and 84 items.

Data prepared in: 0.033505s

Loading user-provided nearest items.

In [10]:
m

Generating candidate set for working with new users.

Finished training in 0.00128s

Class                            : ItemContentRecommender

Schema
------
User ID                          : __implicit_user__
Item ID                          : index
Target                           : None
Additional observation features  : 0
User side features               : []
Item side features               : ['category', 'tags', 'index']

Statistics
----------
Number of observations           : 0
Number of users                  : 0
Number of items                  : 84

Training summary
----------------
Training time                    : 0.0013

Model Parameters
----------------
Model class                      : ItemContentRecommender
threshold                        : 0.0001
similarity_type                  : cosine
training_method                  : auto

Other Settings
--------------
degree_approximation_threshold   : 4096
max_data_passes                  : 4096
max_item_neighborhood_size       : 64
nearest_neighbors_interaction_proportion_threshold : 0.05
seed_item_set_siz

In [11]:
import random

In [12]:
likes = [random.randrange(len(data)) for _ in range(5)]

In [13]:
recs = m.recommend_from_interactions(likes)
recs_list = list(recs['index'])
recommendations = df.iloc[recs_list]
recommendations['score'] = recs['score']

/Users/Devansh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [14]:
df[['name','category', 'tags']].iloc[likes]

,name,category,tags
51,The Orangery,RESTAURANT,"[sightseeing, restaurant]"
18,The Grazing Goat,RESTAURANT,"[restaurant, gastropub]"
53,Rocca di Papa,RESTAURANT,[restaurant]
4,Kensington Gardens,SIGHTS,"[sightseeing, sights, landmark, historicplace,..."
73,John Sandoe Books Ltd,SHOPPING,"[shopping, library]"


In [15]:
recommendations[['score','name','category', 'tags', 'geoCode']]


,score,name,category,tags,geoCode
52,0.470711,Zafferano,RESTAURANT,[restaurant],"{'latitude': 51.499184, 'longitude': -0.157539}"
50,0.470711,The Swan,RESTAURANT,[restaurant],"{'latitude': 51.511513, 'longitude': -0.176875}"
49,0.470711,Khan's,RESTAURANT,[restaurant],"{'latitude': 51.515568, 'longitude': -0.189636}"
48,0.470711,Serpentine Bar & Kitchen,RESTAURANT,[restaurant],"{'latitude': 51.504974, 'longitude': -0.159903}"
43,0.470711,Petrus,RESTAURANT,[restaurant],"{'latitude': 51.49957, 'longitude': -0.156896}"
17,0.470711,The Harwood Arms,RESTAURANT,"[restaurant, gastropub]","{'latitude': 51.48291, 'longitude': -0.19647}"
15,0.470711,Amaya,RESTAURANT,[restaurant],"{'latitude': 51.499287, 'longitude': -0.157143}"
12,0.470711,Restaurant Gordon Ramsay,RESTAURANT,[restaurant],"{'latitude': 51.485832, 'longitude': -0.161966}"
11,0.470711,Le Gavroche,RESTAURANT,[restaurant],"{'latitude': 51.51143, 'longitude': -0.155106}"
8,0.470711,Dinner by Heston Blumenthal,RESTAURANT,[restaurant],"{'latitude': 51.502125, 'longitude': -0.160123}"


In [16]:
data[0]

{'type': 'location',
 'subType': 'POINT_OF_INTEREST',
 'geoCode': {'latitude': 51.496395, 'longitude': -0.171935},
 'name': 'Victoria and Albert Museum',
 'category': 'SIGHTS',
 'tags': ['sightseeing', 'artgallerie', 'museum', 'sights', 'landmark']}